In [8]:
###############
### Imports ###
###############

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
#########################
### Gather Data Files ###
#########################


req = requests.get('https://vehiculeselectriques.gouv.qc.ca/english/rabais/ve-neuf/vehicules-neufs-admissibles.asp')
soup = BeautifulSoup(req.content, 'html.parser')
with open('Eligible vehicles.html') as Eliveh:
    new_soup = BeautifulSoup(Eliveh,'html.parser')
cars = soup.find_all("div", class_="infoVoiture")
fed_cars = new_soup.find('table')
fed_cars = fed_cars.findAll('tr', role='row')[1:]

In [19]:
###########################
### Load into Dataframe ###
###########################


fed_cars_df = pd.DataFrame()

for car in fed_cars:
    car_make = car.find('td', class_='sorting_1').text
    car_model = car.find('td', class_='sorting_3').text
    car_model_year = car.find('td', class_='sorting_2').text
    try:
        car_trim = car.find('td', headers='tbl8').text
        car_category = car.find('td', headers='tbl9').text
        car_rebate = car.find('td', headers='tbl12').text
        car_eligibility = car.find('td', headers = 'tbl16').text
    except:
        car_trim = ''
        car_category = ''
        car_rebate = ''
        car_eligibility = 'Empty'
    fed_cars_df = fed_cars_df.append({'Make':car_make,'Model':car_model, 'Trim':car_trim,'Rebate-QC':car_rebate,'Year-QC':car_model_year,'Category':car_category,'Eligibility':car_eligibility}, ignore_index=True)

cars_df = pd.DataFrame()
makes = ['Audi','BMW','BYD','Chevrolet','Chrysler','Ford','Honda','Hyundai','Jeep','Kia','Lexus','Lincoln','Mazda','Mini','Mitsubishi','Nissan','Polestar','Subaru','Tesla','Toyota','Volkswagen','Volvo']

for car in cars:
    car_category = car.findAll('p')[1].text[10:]
    if car_category in ['All-electric','Plug-in hybrid']:
        car_string = car.find('h2').text
        car_model_year = car.findAll('p')[0].text[15:-2]
        car_rebate = car.find('div', class_='rabais-achat').text[1:-1]
        for make in makes:
            if make in car_string:
                car_name = car_string[:len(make)]
                car_model = car_string[len(make)+1:]
                continue
        cars_df = cars_df.append({'Make':car_name,'Model':car_model,'Rebate-QC':car_rebate,'Year-QC':car_model_year,'Category':car_category}, ignore_index=True)

,Make,Model,Trim,Rebate-QC,Year-QC,Catagory,Eligibility
12,BMW,i3,,,2020,,Empty
27,Ford,*Escape PHEV,,,2022,,Empty
30,Ford,*Escape PHEV,,,2021,,Empty
32,Ford,*Escape PHEV,,,2020,,Empty
41,Hyundai,*Santa Fe PHEV,,,2022,,Empty
42,Hyundai,*Tucson PHEV,,,2022,,Empty
63,Kia,*Sorento PHEV,,,2022,,Empty


In [12]:
######################
### Load from file ###
######################

fed_cars_df = pd.read_csv('fed_cars_df.csv')
cars_df = pd.read_csv('cars_df.csv')